In [ ]:
!unzip /content/CMP_facade_DB_base.zip
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import os
import sys
import numpy as np
from PIL import Image, ImageOps

def get_shape(tensor):
    return tensor.get_shape().as_list()

def batch_norm(*args, **kwargs):

    with tf.name_scope('bn'):
        bn = tf.layers.batch_normalization(*args, **kwargs)

    return bn

def lkrelu(x, slope=0.01):
    return tf.maximum(slope*x, x)


Archive:  /content/CMP_facade_DB_base.zip
   creating: base/
  inflating: base/cmp_b0001.jpg      
  inflating: base/cmp_b0001.png      
  inflating: base/cmp_b0001.xml      
  inflating: base/cmp_b0002.jpg      
  inflating: base/cmp_b0002.png      
  inflating: base/cmp_b0002.xml      
  inflating: base/cmp_b0003.jpg      
  inflating: base/cmp_b0003.png      
  inflating: base/cmp_b0003.xml      
  inflating: base/cmp_b0004.jpg      
  inflating: base/cmp_b0004.png      
  inflating: base/cmp_b0004.xml      
  inflating: base/cmp_b0005.jpg      
  inflating: base/cmp_b0005.png      
  inflating: base/cmp_b0005.xml      
  inflating: base/cmp_b0006.jpg      
  inflating: base/cmp_b0006.png      
  inflating: base/cmp_b0006.xml      
  inflating: base/cmp_b0007.jpg      
  inflating: base/cmp_b0007.png      
  inflating: base/cmp_b0007.xml      
  inflating: base/cmp_b0008.jpg      
  inflating: base/cmp_b0008.png      
  inflating: base/cmp_b0008.xml      
  inflating: base/cmp_b0009

In [ ]:
class Discriminator(object):
    def __init__(self, inputs, is_training, stddev=0.02, center=True, scale=True, reuse=None):
        self._is_training = is_training
        self._stddev = stddev

        with tf.variable_scope('D', initializer=tf.truncated_normal_initializer(stddev=self._stddev), reuse=reuse):
            self._center = center
            self._scale = scale
            self._prob = 0.5
            self._inputs = inputs
            self._discriminator = self._build_discriminator(inputs, reuse=reuse)

    def build_layer(self, name, inputs, k, bn=True, use_dropout=False):
        layer = dict()

        with tf.variable_scope(name):
            layer['filters'] = tf.get_variable('filters', [4, 4, get_shape(inputs)[-1], k])
            layer['conv'] = tf.nn.conv2d(inputs, layer['filters'], strides=[1, 2, 2, 1], padding='SAME')
            layer['bn'] = batch_norm(layer['conv'], center=self._center, scale=self._scale, training=self._is_training) if bn else layer['conv']
            layer['dropout'] = tf.nn.dropout(layer['bn'], self._prob) if use_dropout else layer['bn']
            layer['fmap'] = lkrelu(layer['dropout'], slope=0.2)

        return layer

    def _build_discriminator(self, inputs, reuse=None):
        discriminator = dict()

        discriminator['l1'] = self.build_layer('l1', inputs, 64, bn=False)
        discriminator['l2'] = self.build_layer('l2', discriminator['l1']['fmap'], 128)
        discriminator['l3'] = self.build_layer('l3', discriminator['l2']['fmap'], 256)
        discriminator['l4'] = self.build_layer('l4', discriminator['l3']['fmap'], 512)

        with tf.variable_scope('15'):
            l5 = dict()
            l5['filters'] = tf.get_variable('filters', [4, 4, get_shape(discriminator['l4']['fmap'])[-1], 1])
            l5['conv'] = tf.nn.conv2d(discriminator['l4']['fmap'], l5['filters'], strides=[1, 1, 1, 1], padding="SAME")
            l5['bn'] = batch_norm(l5['conv'], center=self._center, scale=self._scale, training=self._is_training)
            l5['fmap'] = tf.nn.sigmoid(l5['bn'])
            discriminator['l5'] = l5

        return discriminator

In [ ]:
class Generator(object):
    def __init__(self, inputs, is_training, ochan, stddev=0.02, center=True, scale=True, reuse=None):
        self._is_training = is_training
        self._stddev = stddev
        self._ochan = ochan
        with tf.variable_scope('G', initializer=tf.truncated_normal_initializer(stddev=self._stddev), reuse=reuse):
            self._center = center
            self._scale = scale
            self._prob = 0.5
            self._inputs = inputs
            self._encoder = self._build_encoder(inputs)
            self._decoder = self._build_decoder(self._encoder)

    def _build_encoder_layer(self, name, inputs, k, bn=True, use_dropout=False):
        layer = dict()
        with tf.variable_scope(name):
            layer['filters'] = tf.get_variable('filters', [4, 4, get_shape(inputs)[-1], k])
            layer['conv'] = tf.nn.conv2d(inputs, layer['filters'], strides=[1, 2, 2, 1], padding='SAME')
            layer['bn'] = batch_norm(layer['conv'], center=self._center, scale=self._scale, training=self._is_training) if bn else layer['conv']
            layer['dropout'] = tf.nn.dropout(layer['bn'], self._prob) if use_dropout else layer['bn']
            layer['fmap'] = lkrelu(layer['dropout'], slope=0.2)
        return layer

    def _build_encoder(self, inputs):
        encoder = dict()

        
        with tf.variable_scope('encoder'):
            encoder['l1'] = self._build_encoder_layer('l1', inputs, 64, bn=False)
            encoder['l2'] = self._build_encoder_layer('l2', encoder['l1']['fmap'], 128)
            encoder['l3'] = self._build_encoder_layer('l3', encoder['l2']['fmap'], 256)
            encoder['l4'] = self._build_encoder_layer('l4', encoder['l3']['fmap'], 512)
            encoder['l5'] = self._build_encoder_layer('l5', encoder['l4']['fmap'], 512)
            encoder['l6'] = self._build_encoder_layer('l6', encoder['l5']['fmap'], 512)
            encoder['l7'] = self._build_encoder_layer('l7', encoder['l6']['fmap'], 512)
            encoder['l8'] = self._build_encoder_layer('l8', encoder['l7']['fmap'], 512)
        return encoder

    def _build_decoder_layer(self, name, inputs, output_shape_from,use_dropout=False):
        layer = dict()

        with tf.variable_scope(name):
            output_shape = tf.shape(output_shape_from)
            layer['filters'] = tf.get_variable('filters', [4, 4, get_shape(output_shape_from)[-1], get_shape(inputs)[-1]])
            layer['conv'] = tf.nn.conv2d_transpose(inputs, layer['filters'], output_shape=output_shape, strides=[1, 2, 2, 1], padding='SAME')
            layer['bn'] = batch_norm(tf.reshape(layer['conv'], output_shape), center=self._center, scale=self._scale, training=self._is_training)
            layer['dropout'] = tf.nn.dropout(layer['bn'], self._prob) if use_dropout else layer['bn']
            layer['fmap'] = tf.nn.relu(layer['dropout'])
        return layer

    def _build_decoder(self, encoder):
        decoder = dict()

        with tf.variable_scope('decoder'): 
            decoder['dl1'] = self._build_decoder_layer('dl1', encoder['l8']['fmap'], output_shape_from=encoder['l7']['fmap'], use_dropout=True)

            fmap_concat = tf.concat([decoder['dl1']['fmap'], encoder['l7']['fmap']], axis=3)
            decoder['dl2'] = self._build_decoder_layer('dl2', fmap_concat, output_shape_from=encoder['l6']['fmap'], use_dropout=True)

            fmap_concat = tf.concat([decoder['dl2']['fmap'], encoder['l6']['fmap']], axis=3)
            decoder['dl3'] = self._build_decoder_layer('dl3', fmap_concat, output_shape_from=encoder['l5']['fmap'], use_dropout=True)

            fmap_concat = tf.concat([decoder['dl3']['fmap'], encoder['l5']['fmap']], axis=3)
            decoder['dl4'] = self._build_decoder_layer('dl4', fmap_concat, output_shape_from=encoder['l4']['fmap'])

            fmap_concat = tf.concat([decoder['dl4']['fmap'], encoder['l4']['fmap']], axis=3)
            decoder['dl5'] = self._build_decoder_layer('dl5', fmap_concat, output_shape_from=encoder['l3']['fmap'])

            fmap_concat = tf.concat([decoder['dl5']['fmap'], encoder['l3']['fmap']], axis=3)
            decoder['dl6'] = self._build_decoder_layer('dl6', fmap_concat, output_shape_from=encoder['l2']['fmap'])

            fmap_concat = tf.concat([decoder['dl6']['fmap'], encoder['l2']['fmap']], axis=3)
            decoder['dl7'] = self._build_decoder_layer('dl7', fmap_concat, output_shape_from=encoder['l1']['fmap'])

            fmap_concat = tf.concat([decoder['dl7']['fmap'], encoder['l1']['fmap']], axis=3)
            decoder['dl8'] = self._build_decoder_layer('dl8', fmap_concat, output_shape_from=self._inputs)

            with tf.variable_scope('cl9'):
                cl9 = dict()
                cl9['filters'] = tf.get_variable('filters', [4, 4, get_shape(decoder['dl8']['fmap'])[-1], self._ochan])
                cl9['conv'] =  tf.nn.conv2d(decoder['dl8']['fmap'], cl9['filters'], strides=[1, 1, 1, 1], padding='SAME')
                cl9['fmap'] = tf.nn.tanh(cl9['conv'])
                decoder['cl9'] = cl9
        return decoder

    

In [ ]:
print(os.listdir('/content/base/'))

['cmp_b0370.png', 'cmp_b0294.xml', 'cmp_b0154.png', 'cmp_b0044.xml', 'cmp_b0358.jpg', 'cmp_b0054.xml', 'cmp_b0129.jpg', 'cmp_b0355.xml', 'cmp_b0273.xml', 'cmp_b0162.png', 'cmp_b0364.xml', 'cmp_b0116.jpg', 'cmp_b0349.xml', 'cmp_b0011.xml', 'cmp_b0089.png', 'cmp_b0026.jpg', 'cmp_b0107.xml', 'cmp_b0163.jpg', 'cmp_b0111.xml', 'cmp_b0080.png', 'cmp_b0178.xml', 'cmp_b0184.png', 'cmp_b0066.png', 'cmp_b0123.jpg', 'cmp_b0367.xml', 'cmp_b0340.jpg', 'cmp_b0290.xml', 'cmp_b0161.xml', 'cmp_b0319.xml', 'cmp_b0236.xml', 'cmp_b0233.png', 'cmp_b0006.xml', 'cmp_b0338.jpg', 'cmp_b0371.png', 'cmp_b0115.xml', 'cmp_b0035.xml', 'cmp_b0170.png', 'cmp_b0339.png', 'cmp_b0232.jpg', 'cmp_b0077.jpg', 'cmp_b0082.jpg', 'cmp_b0237.jpg', 'cmp_b0257.jpg', 'cmp_b0189.xml', 'cmp_b0299.png', 'cmp_b0205.xml', 'cmp_b0051.jpg', 'cmp_b0357.xml', 'cmp_b0186.xml', 'cmp_b0358.png', 'cmp_b0128.xml', 'cmp_b0378.png', 'cmp_b0112.png', 'cmp_b0088.jpg', 'cmp_b0071.xml', 'cmp_b0028.xml', 'cmp_b0246.xml', 'cmp_b0019.xml', 'cmp_b0231.pn

In [ ]:
path = '/content/base/'

def image_preprocessing(filename, x_size, y_size):
    im = Image.open(filename)
    if filename.endswith('.png'):
        im = im.convert('RGB')

    downsampled_im = ImageOps.fit(im, (x_size, y_size), method=Image.LANCZOS)
    norm_im = np.array(downsampled_im, dtype=np.float32)/255.

    downsampled_im.close()
    im.close()

    return norm_im


if __name__ == '__main__':
    names = []

    for name in os.listdir(path):
        if name.endswith('.jpg'):
            names.append(name[:-4])

    dataset_X = np.zeros((len(names), 256, 256, 3))
    dataset_Y = np.zeros((len(names), 256, 256, 3))

    for i in range(len(names)):
        print(names[i])
        dataset_X[i] = image_preprocessing(os.path.join(path, names[i] + '.jpg'), 256, 256)
        dataset_Y[i] = image_preprocessing(os.path.join(path, names[i] + '.png'), 256, 256)

    np.save('dataset_X.npy', dataset_X)
    np.save('dataset_Y.npy', dataset_Y)



cmp_b0358
cmp_b0129
cmp_b0116
cmp_b0026
cmp_b0163
cmp_b0123
cmp_b0340
cmp_b0338
cmp_b0232
cmp_b0077
cmp_b0082
cmp_b0237
cmp_b0257
cmp_b0051
cmp_b0088
cmp_b0210
cmp_b0294
cmp_b0311
cmp_b0080
cmp_b0004
cmp_b0192
cmp_b0348
cmp_b0278
cmp_b0010
cmp_b0024
cmp_b0218
cmp_b0244
cmp_b0186
cmp_b0164
cmp_b0032
cmp_b0109
cmp_b0212
cmp_b0124
cmp_b0115
cmp_b0147
cmp_b0092
cmp_b0039
cmp_b0176
cmp_b0226
cmp_b0359
cmp_b0013
cmp_b0227
cmp_b0086
cmp_b0193
cmp_b0102
cmp_b0028
cmp_b0187
cmp_b0175
cmp_b0003
cmp_b0158
cmp_b0356
cmp_b0376
cmp_b0374
cmp_b0247
cmp_b0121
cmp_b0238
cmp_b0041
cmp_b0290
cmp_b0323
cmp_b0171
cmp_b0375
cmp_b0031
cmp_b0169
cmp_b0180
cmp_b0350
cmp_b0275
cmp_b0317
cmp_b0233
cmp_b0251
cmp_b0354
cmp_b0368
cmp_b0047
cmp_b0150
cmp_b0044
cmp_b0018
cmp_b0250
cmp_b0225
cmp_b0125
cmp_b0136
cmp_b0048
cmp_b0098
cmp_b0219
cmp_b0001
cmp_b0057
cmp_b0177
cmp_b0099
cmp_b0307
cmp_b0093
cmp_b0334
cmp_b0222
cmp_b0112
cmp_b0287
cmp_b0296
cmp_b0363
cmp_b0295
cmp_b0030
cmp_b0118
cmp_b0253
cmp_b0066
cmp_b0224


In [ ]:
class Pix2pix(object):
    def __init__(self, width, height, ichan, ochan, l1_weight=100., lr=0.0002, beta1=0.5):

        self._is_training = tf.placeholder(tf.bool)
        self._g_inputs = tf.placeholder(tf.float32, [None, width, height, ichan])
        self._d_inputs_a = tf.placeholder(tf.float32, [None, width, height, ichan])
        self._d_inputs_b = tf.placeholder(tf.float32, [None, width, height, ochan])
        self._g = Generator(self._g_inputs, self._is_training, ochan)
        self._real_d = Discriminator(tf.concat([self._d_inputs_a, self._d_inputs_b], axis=3), self._is_training)
        self._fake_d = Discriminator(tf.concat([self._d_inputs_a, self._g._decoder['cl9']['fmap']], axis=3), self._is_training, reuse=tf.AUTO_REUSE)

        self._g_loss = -tf.reduce_mean(tf.log(self._fake_d._discriminator['l5']['fmap'])) + l1_weight * tf.reduce_mean(tf.abs(
                                                                            self._d_inputs_b - self._g._decoder['cl9']['fmap']))
        self._d_loss = -tf.reduce_mean(tf.log(self._real_d._discriminator['l5']['fmap']) + tf.log(1.0 - self._fake_d._discriminator['l5']['fmap']))

        g_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='G')

        with tf.control_dependencies(g_update_ops):
            self._g_train_step = tf.train.AdamOptimizer(lr, beta1=beta1).minimize(self._g_loss,
                    var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='G'))
        
        d_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='D')

        with tf.control_dependencies(d_update_ops):
            self._d_train_step = tf.train.AdamOptimizer(lr, beta1=beta1).minimize(self._d_loss,
                    var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='D'))
            
    def train_step(self, sess, g_inputs, d_inputs_a, d_inputs_b, is_training=True):
        _, dloss_curr = sess.run([self._d_train_step, self._d_loss],
            feed_dict={self._d_inputs_a : d_inputs_a, self._d_inputs_b : d_inputs_b, self._g_inputs : g_inputs, self._is_training : is_training})
        _, gloss_curr = sess.run([self._g_train_step, self._g_loss],
                feed_dict={self._g_inputs : g_inputs, self._d_inputs_a : d_inputs_a,   self._d_inputs_b : d_inputs_b,self._is_training : is_training})
        return (gloss_curr, dloss_curr)

    def sample_generator(self, sess, g_inputs, is_training=False):
        return sess.run(self._g._decoder['cl9']['fmap'], feed_dict={self._g_inputs : g_inputs, self._is_training : is_training})
            

        


In [ ]:
tf.keras.backend.clear_session()
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

iters = 200*400
batch_size = 1

A = np.load('dataset_Y.npy') 
B = np.load('dataset_X.npy') 

with tf.device('/gpu:0'):
    model = Pix2pix(256, 256, ichan=3, ochan=3)

saver = tf.train.Saver()

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(iters):
        a = np.expand_dims(A[step % A.shape[0]], axis=0)
        b = 2. * np.expand_dims(B[step % B.shape[0]], axis=0) - 1. 

        gloss_curr, dloss_curr = model.train_step(sess, a, a, b)
        print('Step %d: G loss: %f | D loss: %f' % (step, gloss_curr, dloss_curr))

        if step % 500 == 0:
            fig = plt.figure()
            fig.set_size_inches(10, 10)
            fig.subplots_adjust(left=0, bottom=0,
                                   right=1, top=1, wspace=0, hspace=0.1)
            p = np.random.permutation(B.shape[0])
            for i in range(0, 81, 3):

                fig.add_subplot(9, 9, i + 1)
                plt.imshow(A[p[i // 3]])
                plt.axis('off')
                fig.add_subplot(9, 9, i + 2)
                plt.imshow((model.sample_generator(sess, np.expand_dims(A[p[i // 3]], axis=0), is_training=True)[0] + 1.) / 2.)
                plt.axis('off')
                fig.add_subplot(9, 9, i +3)
                plt.imshow(B[p[i // 3]])
                plt.axis('off')
            plt.savefig('/content/images/iter_%d.jpg' % step)
            plt.close()
        if step % 3000 == 0:

            save_path = saver.save(sess, "/content/models/model.ckpt")
            print("Model saved in file: %s" % save_path)

            

Streaming output truncated to the last 5000 lines.
Step 1306: G loss: 18.352171 | D loss: 1.457745
Step 1307: G loss: 24.134260 | D loss: 1.475767
Step 1308: G loss: 41.665051 | D loss: 1.466530
Step 1309: G loss: 37.142006 | D loss: 1.458157
Step 1310: G loss: 40.669586 | D loss: 1.474568
Step 1311: G loss: 33.156548 | D loss: 1.480198
Step 1312: G loss: 25.748291 | D loss: 1.451580
Step 1313: G loss: 38.712467 | D loss: 1.447943
Step 1314: G loss: 40.121525 | D loss: 1.477658
Step 1315: G loss: 38.337742 | D loss: 1.456954
Step 1316: G loss: 27.951324 | D loss: 1.468169
Step 1317: G loss: 29.460909 | D loss: 1.455571
Step 1318: G loss: 28.934275 | D loss: 1.467973
Step 1319: G loss: 29.134802 | D loss: 1.477405
Step 1320: G loss: 33.142921 | D loss: 1.449015
Step 1321: G loss: 25.128792 | D loss: 1.451960
Step 1322: G loss: 44.536415 | D loss: 1.472603
Step 1323: G loss: 29.887190 | D loss: 1.473131
Step 1324: G loss: 42.319950 | D loss: 1.518323
Step 1325: G loss: 26.139912 | D loss

KeyboardInterrupt: ignored

In [ ]:
# example of loading a pix2pix model and using it for one-off image translation
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import load
from numpy import expand_dims
from matplotlib import pyplot
 
# load an image
def load_image(filename, size=(256,256)):
	# load image with the preferred size
	pixels = load_img(filename, target_size=size)
	# convert to numpy array
	pixels = img_to_array(pixels)
	# scale from [0,255] to [-1,1]
	pixels = (pixels - 127.5) / 127.5
	# reshape to 1 sample
	pixels = expand_dims(pixels, 0)
	return pixels
 
# load source image
src_image = load_image('satellite.jpg')
print('Loaded', src_image.shape)
# load model
model = load_model('model_109600.h5')
# generate image from source
gen_image = model.predict(src_image)
# scale from [-1,1] to [0,1]
gen_image = (gen_image + 1) / 2.0
# plot the image
pyplot.imshow(gen_image[0])
pyplot.axis('off')
pyplot.show()